# Reading Mortgage data

In [1]:
import pandas as pd
import numpy as np

In [2]:

# Path to the uploaded CSV file
csv_path = '/var/R/home/aa630798/Test_mortgages.csv'
# Read the CSV file with a specified delimiter
mortgages_df = pd.read_csv(csv_path, delimiter=',')
# Set pandas to display all rows
pd.set_option('display.max_rows', None)

#print(mortgages_df)


# variables of unique values from data set Mortgages

In [3]:
unique_rvps = mortgages_df['Looptijd Rentevastperiode 1'].unique() # indicates the values of fixed interest periods in months 
unique_aflosvorm = mortgages_df['Aflosvorm'].unique() # types of repayment like annuity, linear, etc.
print("Unieke Rentevaste Periodes:", unique_rvps)
print("Unieke Aflosvormen:", unique_aflosvorm)

Unieke Rentevaste Periodes: [ 12 120 360]
Unieke Aflosvormen: ['Aflossingsvrij' 'Lineair' 'Annuiteit' 'Levensverzekering']


# Reading Data from Data set

In [31]:
num_steps = 360

issue_dt = pd.to_datetime(mortgages_df['Datum Ingang Leningdeel']) # Start date of the loan part
mat_dt = pd.to_datetime(mortgages_df['Datum Eind Leningdeel']) # End date of the loan part
reprice_dt = pd.to_datetime(mortgages_df['Datum Eind RVP']) # End date of the fixed interest period
fixed_per = mortgages_df['Looptijd Rentevastperiode 1'] # Fixed interest period in months
orig_principal = mortgages_df['Hoofdsom Oorspronkelijk'] # Original principal amount
curr_principal = mortgages_df['Hoofdsom Restant'] # Current remaining principal amount
int_rate = mortgages_df['Rente Nominaal'] # Nominal interest rate
red_types = mortgages_df['Aflosvorm'] # Types of repayment
remaining_rvp = mortgages_df['Looptijd Rentevastperiode 1']  # Remaining fixed interest period in months
principal_savings = mortgages_df['Bedrag Banksparen'] + mortgages_df['Bedrag Spaardepot']  # Savings amount for bank savings and savings deposit


#print("issue_dt:", issue_dt)
#print("mat_dt:", mat_dt)
#print("reprice_dt:", reprice_dt)
#print("fixed_per:", fixed_per)
#print("orig_principal:", orig_principal)
#print("curr_principal:", curr_principal)
#print("int_rate:", int_rate)
#print("red_types:", red_types)
#print("remaining_rvp:", remaining_rvp)
#print("principal_savings:", principal_savings)

#  Creating empty arrays to store data 

In [32]:
num_morts = len(mortgages_df)
tot_principal = np.zeros((num_morts, num_steps))
tot_redemptions = np.zeros((num_morts, num_steps))
tot_resets = np.zeros((num_morts, num_steps))
tot_interest = np.zeros((num_morts, num_steps))
tot_count = np.zeros((num_morts, num_steps))


#  Initializations for different mortgage combinations

In [33]:
mort_types = [''] * num_morts  # Array to store types of mortgages
count_mort_types = np.zeros(num_morts) # Array to count the number of each type of mortgage
tot_fltr = np.zeros(len(mortgages_df), dtype=bool)  # Boolean array to filter mortgages based on conditions


In [34]:
now_dt = pd.Timestamp('2023-03-31').toordinal()  # Spotdate

# Calling the MortSched function

In [35]:
mortgages_df = mortgages_df.sort_values(by=['Aflosvorm', 'Looptijd Rentevastperiode 1']).reset_index(drop=True)

def MortSched(now_dt, issue_dt, mat_dt, reprice_dt, remaining_rvp, red_type, fixed_per, principal_savings, curr_principal, int_rate):
    num_steps = 360
    num_mortgages = len(issue_dt)
    mort_dates = np.zeros((num_mortgages, num_steps))
    interest = np.zeros((num_mortgages, num_steps))
    principal = np.zeros((num_mortgages, num_steps))
    resets = np.zeros((num_mortgages, num_steps))
    mort_count = np.zeros((num_mortgages, num_steps))
    
    for i in range(num_mortgages):
        term = (mat_dt.iloc[i] - issue_dt.iloc[i]).days // 30
        monthly_rate = int_rate.iloc[i] / 12 / 100
        remaining_principal = curr_principal.iloc[i]
        
        for j in range(min(term, num_steps)):
            interest[i, j] = remaining_principal * monthly_rate
            
            if red_type == 'Annuiteit':
                annuity_payment = remaining_principal * (monthly_rate / (1 - (1 + monthly_rate) ** -term))
                principal[i, j] = annuity_payment - interest[i, j]
            elif red_type == 'Linear':
                principal[i, j] = remaining_principal / term
            elif red_type == 'Aflossingsvrij':
                principal[i, j] = 0
            elif red_type == 'Levensverzekering':
                # Simpele benadering, aanname dat levensverzekering dezelfde logica volgt als annuiteit
                annuity_payment = remaining_principal * (monthly_rate / (1 - (1 + monthly_rate) ** -term))
                principal[i, j] = annuity_payment - interest[i, j]
            else:
                principal[i, j] = 0  # Voeg hier andere aflosvormen toe indien nodig
            
            remaining_principal -= principal[i, j]
            mort_count[i, j] = 1
    
    return mort_dates, interest, principal, resets, mort_count

# For loop followed by cashflow results

In [37]:
def create_mortgage_flows():
    for red_idx in range(len(unique_aflosvorm)):
        for rvp_idx in range(len(unique_rvps)):
            aflosvorm = unique_aflosvorm[red_idx]
            rvp = unique_rvps[rvp_idx]
            
            fltr = (mortgages_df['Looptijd Rentevastperiode 1'] == rvp) & (mortgages_df['Aflosvorm'] == aflosvorm)
            if fltr.sum() == 0:
                continue

            print(f"Processing {aflosvorm} with RVP {rvp}...")

            for idx in mortgages_df[fltr].index:
                mort_dates, interest, principal, resets, mort_count = MortSched(
                    now_dt, 
                    issue_dt[idx:idx+1], 
                    mat_dt[idx:idx+1], 
                    reprice_dt[idx:idx+1], 
                    remaining_rvp[idx:idx+1], 
                    aflosvorm, 
                    fixed_per[idx:idx+1], 
                    principal_savings[idx:idx+1], 
                    curr_principal[idx:idx+1], 
                    int_rate[idx:idx+1]
                )
                
                max_steps = min(fixed_per[idx], 360)  # Limiteer tot de rentevaste periode of 360 maanden
                
                tot_count[idx, :max_steps] = mort_count[0, :max_steps]
                tot_resets[idx, :max_steps] = resets[0, :max_steps]
                tot_redemptions[idx, :max_steps] = -np.diff(np.append([0], np.sum(principal, axis=0)[:max_steps]))
                tot_principal[idx, :max_steps] = np.sum(principal, axis=0)[:max_steps]
                tot_interest[idx, :max_steps] = np.sum(interest[:, :max_steps], axis=0)
                
                mort_types[idx] = f"{aflosvorm}_{rvp}"

                if abs(np.sum(curr_principal[idx:idx+1]) - np.sum(-np.diff(np.sum(principal, axis=0)))) > 1:
                    print(f"Principal mismatch for index {idx}: {abs(np.sum(curr_principal[idx:idx+1]) - np.sum(-np.diff(np.sum(principal, axis=0))))}")

    return tot_principal, tot_redemptions, tot_interest

def print_cashflows(tot_principal, tot_redemptions, tot_interest):
    for i in range(num_morts):
        print(f"Hypotheek {i} met aflosvorm en looptijd {mort_types[i]}:")
        max_steps = min(fixed_per[i], 360)
        for month in range(max_steps):
            print(f"  Cashflow maand {month + 1}: {tot_principal[i, month] + tot_interest[i, month]}")

tot_principal, tot_redemptions, tot_interest = create_mortgage_flows()
print_cashflows(tot_principal, tot_redemptions, tot_interest)

Processing Aflossingsvrij with RVP 12...
Principal mismatch for index 0: 185000.0
Processing Aflossingsvrij with RVP 120...
Principal mismatch for index 1: 31800.0
Principal mismatch for index 2: 118200.0
Principal mismatch for index 3: 51000.0
Principal mismatch for index 4: 158500.0
Principal mismatch for index 5: 18000.0
Processing Lineair with RVP 12...
Principal mismatch for index 16: 103661.89
Processing Annuiteit with RVP 120...
Principal mismatch for index 6: 226616.11896573103
Principal mismatch for index 7: 119081.91908481764
Principal mismatch for index 8: 117411.9775437321
Principal mismatch for index 9: 93041.86908251092
Principal mismatch for index 10: 16131.516029473933
Principal mismatch for index 11: 107186.49999534767
Principal mismatch for index 12: 93131.97566559564
Processing Annuiteit with RVP 360...
Principal mismatch for index 13: 158970.55937826924
Processing Levensverzekering with RVP 120...
Principal mismatch for index 14: 31123.03802069657
Principal mismatch